# Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset

In [2]:
dataset = pd.read_csv('50_Startups.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# dataset
# X.size

# Taking care of missing data

In [3]:
#from sklearn.preprocessing import Imputer
#imputer = Imputer(missing_values="NaN",strategy="mean",axis=0)
#imputer = imputer.fit(X[:,1:3])
#X[:,1:3] = imputer.transform(X[:,1:3])

# Encoding categorical data

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:,-1] = labelencoder_X.fit_transform(X[:,-1])
onehotencoder = OneHotEncoder(categorical_features=[3])
X = onehotencoder.fit_transform(X).toarray()
# labelencoder_y = LabelEncoder()
# y = labelencoder_y.fit_transform(y)
# np.set_printoptions(precision=3)
# np.set_printoptions(precision=1)
# X

In [5]:
# Avoiding dummy variable trap

In [6]:
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
#X_train.size

# Feature Scaling

In [8]:
#from sklearn.preprocessing import StandardScaler
#sc_X = StandardScaler()
#X_train = sc_X.fit_transform(X_train)
#X_test = sc_X.transform(X_test)
#sc_y = StandardScaler()
#y_train = sc_y.fit_transform(y_train)

# Fitting multiple linear regression to the training set

In [9]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

# Predicting the test set results

In [10]:
y_pred = regressor.predict(X_test)
# y_pred

# Building optimal model using backward elimination

In [11]:
import statsmodels.formula.api as sm
X = np.append(arr= np.ones((50,1)).astype(int), values= X, axis= 1)
# X
#X_opt = X[:, [0, 1, 2, 3, 4, 5]]
#regressor_ols = sm.OLS(endog= y, exog= X_opt).fit()
#regressor_ols.summary()
#X_opt = X[:, [0, 1, 3, 4, 5]]
#regressor_ols = sm.OLS(endog= y, exog= X_opt).fit()
#regressor_ols.summary()
#X_opt = X[:, [0, 3, 4, 5]]
#regressor_ols = sm.OLS(endog= y, exog= X_opt).fit()
#regressor_ols.summary()
#X_opt = X[:, [0, 3, 5]]
#regressor_ols = sm.OLS(endog= y, exog= X_opt).fit()
#regressor_ols.summary()
#X_opt = X[:, [0, 3]]
#regressor_ols = sm.OLS(endog= y, exog= X_opt).fit()
#regressor_ols.summary()

# Automatic backward elimination with p-values only

In [43]:
import statsmodels.formula.api as sm
def backwardElimination(x, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x
 
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)

# Automatic backward elimination with p-values and adjusted R squared

In [44]:
import statsmodels.formula.api as sm
def backwardElimination(x, SL):
    numVars = len(x[0])
    temp = np.zeros((50,6)).astype(int)
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        adjR_before = regressor_OLS.rsquared_adj.astype(float)
        if maxVar > SL:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    temp[:,j] = x[:, j]
                    x = np.delete(x, j, 1)
                    tmp_regressor = sm.OLS(y, x).fit()
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
                    if (adjR_before >= adjR_after):
                        x_rollback = np.hstack((x, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (regressor_OLS.summary())
                        return x_rollback
                    else:
                        continue
    regressor_OLS.summary()
    return x
 
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Sun, 21 Oct 2018   Prob (F-statistic):           2.16e-31
Time:                        00:34:01   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.0